# Lean 5 - Mode Tactique

## Introduction

Jusqu'ici, nous avons construit des preuves en ecrivant directement des **termes de preuve**. Cette approche, bien que fondamentale, peut devenir fastidieuse pour des preuves complexes.

Le **mode tactique** offre une approche alternative : au lieu de construire le terme de preuve directement, on manipule des **buts** (goals) de maniere interactive jusqu'a ce que la preuve soit complete.

### Objectifs pedagogiques

1. Comprendre le mode tactique et la notion de but
2. Maitriser les tactiques de base : `apply`, `exact`, `intro`, `rfl`
3. Gerer le contexte avec `have`, `let`, `show`
4. Utiliser l'analyse par cas avec `cases` et `split`
5. Reecrire avec `rw` et simplifier avec `simp`
6. Structurer les preuves avec points et combinateurs

### Prerequis

- Notebooks **Lean-1** a **Lean-4** completes
- Comprendre la construction de preuves par termes

### Duree estimee : 50-60 minutes

---

## Le Paradigme Tactique

### Termes vs Tactiques

| Mode terme | Mode tactique |
|------------|---------------|
| Construction directe | Construction guidee |
| Top-down (resultat -> composants) | Bottom-up (buts -> sous-buts) |
| Compact | Lisible etape par etape |
| Difficulte : voir la structure globale | Difficulte : verbeux |

Le mot-cle `by` introduit le mode tactique.

## 1. Introduction au Mode Tactique

### 1.1 Le mot-cle `by`

On entre en mode tactique avec `by`. L'etat de preuve contient :
- Le **but** (goal) : la proposition a prouver
- Le **contexte** : les hypotheses disponibles

In [ ]:
-- Meme theoreme, deux styles

-- Style terme
theorem impl_refl_term (p : Prop) : p -> p :=
  fun hp => hp

-- Style tactique
theorem impl_refl_tactic (p : Prop) : p -> p := by
  intro hp      -- Introduire l'hypothese hp : p
  exact hp      -- Fournir exactement hp comme preuve

#check impl_refl_term   -- impl_refl_term : (p : Prop) -> p -> p
#check impl_refl_tactic -- meme type

### 1.2 L'etat de preuve

A chaque etape, Lean affiche l'etat de preuve :
- `context` : les hypotheses disponibles (au-dessus de la barre)
- `goal` : ce qu'il reste a prouver (en-dessous de la barre)

```
p : Prop
hp : p
⊢ p          <-- le but courant
```

In [ ]:
-- Exemple avec plusieurs buts
theorem and_intro_tactic (p q : Prop) (hp : p) (hq : q) : p /\ q := by
  -- But initial : ⊢ p /\ q
  constructor   -- Divise en deux buts : ⊢ p et ⊢ q
  -- Premier but : ⊢ p
  exact hp
  -- Deuxieme but : ⊢ q
  exact hq

## 2. Tactiques de Base

### 2.1 `exact` : fournir le terme exact

Si on a un terme `t` de type exactement egal au but, `exact t` ferme le but.

In [ ]:
theorem exact_example (p : Prop) (hp : p) : p := by
  exact hp   -- hp a exactement le type du but

-- Avec un calcul
theorem exact_calc : 2 + 2 = 4 := by
  exact rfl  -- rfl prouve une egalite par calcul

### 2.2 `intro` : introduction d'hypotheses

`intro` fonctionne pour :
- Les implications `P -> Q` : introduit une hypothese de type `P`
- Les forall `\forall x, P x` : introduit une variable `x`

In [ ]:
-- Introduction d'implication
theorem intro_impl (p q : Prop) : p -> q -> p := by
  intro hp     -- hp : p ajoute au contexte, but devient q -> p
  intro hq     -- hq : q ajoute, but devient p
  exact hp

-- intro multiple
theorem intro_multi (p q : Prop) : p -> q -> p := by
  intro hp hq  -- Introduit les deux d'un coup
  exact hp

-- Introduction de forall
theorem intro_forall : forall n : Nat, n + 0 = n := by
  intro n      -- n : Nat ajoute au contexte
  rfl          -- Preuve par reflexivite (calcul)

### 2.3 `apply` : appliquer un lemme/hypothese

Si on a `h : A -> B` et le but est `B`, alors `apply h` change le but en `A`.

In [ ]:
-- apply reduit le but
theorem apply_example (p q r : Prop)
  (hpq : p -> q) (hqr : q -> r) (hp : p) : r := by
  apply hqr    -- But : r devient q (car hqr : q -> r)
  apply hpq    -- But : q devient p
  exact hp     -- hp a type p

-- apply avec plusieurs arguments
theorem apply_multi (p q r : Prop)
  (hpqr : p -> q -> r) (hp : p) (hq : q) : r := by
  apply hpqr   -- Cree deux buts : p et q
  exact hp
  exact hq

### 2.4 `assumption` : chercher dans le contexte

La tactique `assumption` cherche automatiquement dans le contexte une hypothese qui correspond exactement au but courant. C'est un raccourci pratique quand la preuve est deja dans les hypotheses.

In [ ]:
-- assumption trouve automatiquement l'hypothese
theorem assumption_example (p q : Prop) (hp : p) (hq : q) : p := by
  assumption   -- Trouve hp : p dans le contexte

-- Utile quand on ne veut pas nommer
theorem assumption_chain (p q r : Prop)
  (hpq : p -> q) (hqr : q -> r) (hp : p) : r := by
  apply hqr
  apply hpq
  assumption

### 2.5 `rfl` : reflexivite

La tactique `rfl` (reflexivite) prouve les egalites triviales ou calculables. Lean reduit les deux cotes de l'egalite et verifie qu'ils sont identiques. Fonctionne pour `2 + 2 = 4`, `n + 0 = n`, etc.

In [ ]:
-- rfl ferme les buts de la forme t = t (apres reduction)
theorem rfl_examples : 2 + 3 = 5 := by rfl

theorem rfl_list : [1, 2, 3].length = 3 := by rfl

-- rfl echoue si pas definitionnellement egal
-- theorem fail_rfl (n : Nat) : n + 0 = n := by rfl  -- ERREUR

## 3. Gestion du Contexte

Les tactiques de cette section permettent de manipuler le contexte de preuve : ajouter des hypotheses intermediaires, modifier le but, ou reorganiser les variables.

In [ ]:
-- have introduit un fait intermediaire
theorem have_example (p q r : Prop)
  (hpq : p -> q) (hqr : q -> r) (hp : p) : r := by
  have hq : q := hpq hp   -- Introduit hq : q
  have hr : r := hqr hq   -- Introduit hr : r
  exact hr

-- have avec preuve tactique
theorem have_tactic (p q : Prop) (hpq : p -> q) (hp : p) : q /\ p := by
  have hq : q := by
    apply hpq
    exact hp
  exact ⟨hq, hp⟩

### 3.2 `let` : definitions locales

La tactique `let` introduit une definition locale (variable avec sa valeur) dans le contexte. Utile pour nommer des expressions complexes reutilisees plusieurs fois.

In [ ]:
-- let pour des valeurs calculees
theorem let_example (n : Nat) : (n + 1) * 2 = 2 * n + 2 := by
  let m := n + 1     -- m = n + 1 visible dans le contexte
  calc m * 2 = 2 * m         := Nat.mul_comm m 2
           _ = 2 * (n + 1)   := rfl
           _ = 2 * n + 2 * 1 := Nat.mul_add 2 n 1
           _ = 2 * n + 2     := by ring

### 3.3 `show` : annoter le but

La tactique `show` permet d'indiquer explicitement le but que l'on prouve. Utile pour la clarte ou quand Lean n'infere pas correctement le type attendu.

In [ ]:
-- show clarifie le but (utile pour la lisibilite)
theorem show_example (p q : Prop) (hp : p) (hq : q) : q /\ p := by
  constructor
  show q        -- Annonce qu'on prouve q
  exact hq
  show p        -- Annonce qu'on prouve p
  exact hp

### 3.4 `revert` : remettre dans le but

La tactique `revert` est l'inverse de `intro` : elle deplace une hypothese du contexte vers le but, transformant `h : P |- Q` en `|- P -> Q`.

In [ ]:
-- revert est l'inverse de intro
theorem revert_example (p q : Prop) : p -> q -> p := by
  intro hp hq
  -- Contexte : hp : p, hq : q. But : p
  revert hq      -- But devient : q -> p
  intro _        -- Reintroduit (avec nom anonyme)
  exact hp

## 4. Tactiques pour la Logique

### 4.1 `constructor` : introduction de structures

In [ ]:
-- constructor pour And (conjonction)
theorem and_tactic (p q : Prop) (hp : p) (hq : q) : p /\ q := by
  constructor    -- Divise en deux buts
  · exact hp     -- Premier but avec point
  · exact hq     -- Deuxieme but

-- constructor pour Iff (equivalence)
theorem iff_tactic (p q : Prop) (hpq : p -> q) (hqp : q -> p) : p <-> q := by
  constructor
  · exact hpq
  · exact hqp

-- constructor pour Exists
theorem exists_tactic : \exists n : Nat, n > 5 := by
  constructor           -- But : 6 > 5 (Lean devine le temoin?)
  -- Non, utiliser use plutot

### 4.2 `cases` : analyse par cas

La tactique `cases` decompose une hypothese selon sa structure. Pour `Or` elle cree deux buts, pour `And` elle extrait les composantes, pour `Exists` elle extrait le temoin.

In [ ]:
-- cases pour Or (disjonction)
theorem or_cases (p q r : Prop)
  (hpq : p \/ q) (hpr : p -> r) (hqr : q -> r) : r := by
  cases hpq with
  | inl hp => exact hpr hp    -- Cas p
  | inr hq => exact hqr hq    -- Cas q

-- cases pour And (destructure)
theorem and_cases (p q : Prop) (hpq : p /\ q) : q /\ p := by
  cases hpq with
  | intro hp hq => exact ⟨hq, hp⟩

-- cases pour Exists
theorem exists_cases (P : Nat -> Prop)
  (h : \exists n, P n) : \exists n, P n := by
  cases h with
  | intro w hw => exact ⟨w, hw⟩

### 4.3 `left` et `right` : introduction de Or

Les tactiques `left` et `right` choisissent quel cote d'une disjonction prouver. `left` transforme le but `P \/ Q` en `P`, `right` le transforme en `Q`.

In [ ]:
-- Choisir le cote de la disjonction
theorem left_example (p q : Prop) (hp : p) : p \/ q := by
  left           -- But devient p
  exact hp

theorem right_example (p q : Prop) (hq : q) : p \/ q := by
  right          -- But devient q
  exact hq

### 4.4 `contradiction` : detecter l'absurdite

La tactique `contradiction` cherche automatiquement une contradiction dans le contexte (ex: `h : P` et `hn : Not P`). Si trouvee, le but est resolu.

In [ ]:
-- contradiction trouve p et \neg p dans le contexte
theorem contradiction_example (p q : Prop) (hp : p) (hnp : \neg p) : q := by
  contradiction

-- Fonctionne aussi avec False
theorem false_contradiction (p : Prop) (hf : False) : p := by
  contradiction

## 5. Reecriture avec `rw`

### 5.1 Utilisation de base

`rw [h]` remplace le cote gauche de l'egalite `h` par le cote droit dans le but.

In [ ]:
-- Reecriture simple
theorem rw_example (a b c : Nat) (h : a = b) : a + c = b + c := by
  rw [h]         -- Remplace a par b
  -- But : b + c = b + c
  rfl

-- Reecriture inverse avec <-
theorem rw_rev (a b : Nat) (h : a = b) : b = a := by
  rw [<- h]      -- Remplace b par a (inverse)
  -- But : a = a
  rfl

-- Enchainer les reecritures
theorem rw_chain (a b c : Nat) (h1 : a = b) (h2 : b = c) : a = c := by
  rw [h1, h2]    -- Applique h1 puis h2

### 5.2 Reecriture avec lemmes de la bibliotheque

On peut utiliser `rw` avec des lemmes nommes de la bibliotheque standard. Par exemple `rw [Nat.add_comm]` applique la commutativite de l'addition.

In [ ]:
-- Utiliser les lemmes de Nat
theorem rw_lib (n : Nat) : n + 0 = n := by
  rw [Nat.add_zero]   -- Applique n + 0 = n

theorem rw_comm (a b : Nat) : a + b = b + a := by
  rw [Nat.add_comm]

-- Preuve plus complexe
theorem rw_complex (a b c : Nat) : (a + b) + c = (a + c) + b := by
  rw [Nat.add_assoc]          -- a + (b + c)
  rw [Nat.add_comm b c]       -- a + (c + b)
  rw [<- Nat.add_assoc]       -- (a + c) + b

### 5.3 Reecriture dans le contexte avec `at`

In [ ]:
-- rw at h reecrit dans l'hypothese h
theorem rw_at (a b : Nat) (h : a + 0 = b) : a = b := by
  rw [Nat.add_zero] at h   -- h devient a = b
  exact h

-- rw at * reecrit partout
theorem rw_at_star (a b c : Nat) (h1 : a = b) (h2 : a + c = 10) : b + c = 10 := by
  rw [h1] at *    -- Remplace a par b dans h2 et le but
  exact h2

## 6. Simplification avec `simp`

### 6.1 Utilisation de base

`simp` applique automatiquement un ensemble de regles de simplification.

In [ ]:
-- simp simplifie automatiquement
theorem simp_example (n : Nat) : n + 0 = n := by
  simp   -- Connait n + 0 = n

-- simp sur des listes
theorem simp_list : [1, 2] ++ [3] = [1, 2, 3] := by
  simp

-- simp ne resout pas tout
-- theorem simp_fail (n m : Nat) : n + m = m + n := by simp  -- FAIL

### 6.2 Options de simp

In [ ]:
-- simp avec lemmes additionnels
theorem simp_with (a b : Nat) (h : a = b) : a + 1 = b + 1 := by
  simp [h]   -- Ajoute h aux regles

-- simp only : restreint les regles
theorem simp_only_example (n : Nat) : n + 0 + 0 = n := by
  simp only [Nat.add_zero]

-- simp_all : simplifie aussi le contexte
theorem simp_all_example (a b : Nat) (h : a + 0 = b) : a = b := by
  simp_all

### 6.3 L'attribut `@[simp]`

On peut ajouter ses propres lemmes a la base de simp.

In [ ]:
-- Definir un lemme simp
@[simp] theorem my_simp_lemma (n : Nat) : n * 1 = n := Nat.mul_one n

-- Maintenant simp l'utilise automatiquement
theorem use_my_simp (a : Nat) : a * 1 + 0 = a := by
  simp   -- Utilise my_simp_lemma et add_zero

## 7. Structuration des Preuves

### 7.1 Points (bullets) pour focaliser

In [ ]:
-- Les points separent les sous-buts
theorem bullets_example (p q : Prop) (hp : p) (hq : q) : p /\ q /\ p := by
  constructor
  · exact hp              -- Premier but : p
  · constructor           -- Deuxieme but : q /\ p
    · exact hq            -- Sous-but : q
    · exact hp            -- Sous-but : p

### 7.2 `case` pour nommer les cas

In [ ]:
-- case nomme explicitement les branches
theorem case_example (p q : Prop) (hpq : p \/ q) : q \/ p := by
  cases hpq
  case inl hp =>
    right
    exact hp
  case inr hq =>
    left
    exact hq

### 7.3 Combinateurs `;` et `<;>`

In [ ]:
-- ; enchaine les tactiques
theorem semi_example (p : Prop) (hp : p) : p := by
  exact hp

-- <;> applique a tous les buts generes
theorem all_goals (p : Prop) (hp : p) : p /\ p /\ p := by
  constructor <;> (try constructor) <;> exact hp

-- Plus lisible avec repeat
theorem repeat_example (p : Prop) (hp : p) : p /\ p /\ p := by
  refine ⟨?_, ?_, ?_⟩ <;> exact hp

## 8. Tactiques Avancees

### 8.1 `induction` : preuve par recurrence

In [ ]:
-- Recurrence sur Nat
theorem sum_formula (n : Nat) : 2 * (List.range n).sum = n * (n - 1) := by
  induction n with
  | zero => simp
  | succ k ih =>
    simp [List.range_succ, List.sum_append]
    sorry  -- Preuve complete omise pour la brievete

-- Exemple plus simple : 0 + n = n
theorem zero_add (n : Nat) : 0 + n = n := by
  induction n with
  | zero => rfl
  | succ k ih =>
    simp [Nat.add_succ, ih]

### 8.2 `decide` : propositions decidables

In [ ]:
-- decide resout les propositions decidables
theorem decide_example : 3 < 5 := by decide
theorem decide_bool : true && false = false := by decide
theorem decide_prime : Nat.Prime 7 := by decide

### 8.3 `omega` : arithmetique lineaire

In [ ]:
-- omega resout automatiquement l'arithmetique lineaire
theorem omega_example (n m : Nat) (h : n < m) : n + 1 <= m := by
  omega

theorem omega_complex (a b c : Nat)
  (h1 : a + b < c) (h2 : c < 2 * a + b) : a > 0 := by
  omega

### 8.4 `ring` : algebre

In [ ]:
-- ring resout les egalites polynomiales
theorem ring_example (a b : Int) : (a + b) * (a + b) = a*a + 2*a*b + b*b := by
  ring

theorem ring_nat (n : Nat) : (n + 1) * (n + 1) = n * n + 2 * n + 1 := by
  ring

## 9. Melange Termes et Tactiques

### 9.1 `by` dans un terme

In [ ]:
-- Utiliser by au milieu d'un terme
theorem hybrid_proof (p q : Prop) (hp : p) (hq : q) : p /\ q :=
  ⟨by exact hp, by assumption⟩

-- Preuve partiellement tactique
def safeDivide (a b : Nat) (h : b \ne 0) : Nat :=
  a / b   -- La preuve que b \ne 0 est portee par h

### 9.2 Termes dans les tactiques

In [ ]:
-- exact accepte n'importe quel terme
theorem term_in_tactic (p q : Prop) (hp : p) (hq : q) : p /\ q := by
  exact ⟨hp, hq⟩   -- Terme structure

-- show ... from ... dans by
theorem show_from (p q : Prop) (hp : p) (hq : q) : p /\ q := by
  show p /\ q
  exact And.intro hp hq

## 10. Exercices

### Exercice 1 : Preuve tactique de l'associativite de And

In [ ]:
variable (p q r : Prop)

theorem and_assoc_tactic : (p /\ q) /\ r <-> p /\ (q /\ r) := by
  sorry

### Exercice 2 : Utiliser rw pour une preuve arithmetique

In [ ]:
theorem arith_rw (a b : Nat) : (a + b) + (b + a) = 2 * (a + b) := by
  sorry

### Exercice 3 : Analyse par cas avec Or

In [ ]:
theorem or_distrib (p q r : Prop) :
  p /\ (q \/ r) <-> (p /\ q) \/ (p /\ r) := by
  sorry

## Solutions des exercices

In [ ]:
-- Solution exercice 1
theorem and_assoc_sol (p q r : Prop) : (p /\ q) /\ r <-> p /\ (q /\ r) := by
  constructor
  · intro h
    cases h with
    | intro hpq hr =>
      cases hpq with
      | intro hp hq =>
        exact ⟨hp, hq, hr⟩
  · intro h
    cases h with
    | intro hp hqr =>
      cases hqr with
      | intro hq hr =>
        exact ⟨⟨hp, hq⟩, hr⟩

-- Solution exercice 2
theorem arith_rw_sol (a b : Nat) : (a + b) + (b + a) = 2 * (a + b) := by
  rw [Nat.add_comm b a]   -- (a + b) + (a + b)
  rw [<- Nat.two_mul]     -- 2 * (a + b)

-- Solution exercice 3
theorem or_distrib_sol (p q r : Prop) :
  p /\ (q \/ r) <-> (p /\ q) \/ (p /\ r) := by
  constructor
  · intro ⟨hp, hqr⟩
    cases hqr with
    | inl hq => left; exact ⟨hp, hq⟩
    | inr hr => right; exact ⟨hp, hr⟩
  · intro h
    cases h with
    | inl hpq => exact ⟨hpq.1, Or.inl hpq.2⟩
    | inr hpr => exact ⟨hpr.1, Or.inr hpr.2⟩

## Resume des Tactiques

| Tactique | Usage | Exemple |
|----------|-------|--------|
| `exact t` | Fournir le terme exact | `exact hp` |
| `intro x` | Introduire hypothese/variable | `intro hp hq` |
| `apply h` | Appliquer un lemme | `apply Nat.add_comm` |
| `assumption` | Chercher dans contexte | `assumption` |
| `rfl` | Reflexivite/calcul | `rfl` |
| `constructor` | Introduction structure | `constructor` |
| `cases h` | Analyse par cas | `cases h with ...` |
| `left`/`right` | Choisir branche Or | `left` |
| `have h : P := ...` | Lemme intermediaire | `have hq := ...` |
| `rw [h]` | Reecriture | `rw [Nat.add_zero]` |
| `simp` | Simplification auto | `simp [h]` |
| `contradiction` | Detecter absurdite | `contradiction` |
| `induction n` | Recurrence | `induction n with ...` |
| `omega` | Arithmetique lineaire | `omega` |
| `ring` | Algebre polynomiale | `ring` |

### Prochaine etape

Dans le notebook **Lean-6-Mathlib-Essentials**, nous explorerons **Mathlib4**, la bibliotheque mathematique communautaire de Lean 4, avec ses tactiques puissantes et sa vaste collection de theoremes.

---

*Notebook base sur "TP - Z3 - Tweety - Lean.pdf" Section VI.B.4 et adapte pour Lean 4*